In [1]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd 
# 重要信息，此程序需要再user_tag_recom_main.py 和 user_song_recom_main.py后运行
# 通过用户进行歌曲的歌曲相似度推荐 获取的其中的相似歌曲
mongomus = mus.UserSongRecomAfterSongSimilarity(
    user_id="1a78ac10b67911ed95dd00155dadb10b")
# 1a78ac10b67911ed95dd00155dadb10b
# mongomus.makeRecomBySongSimilarityAnswer(limit="ONE")
# 创建程序进程

In [2]:
# 程序从这里开始
# ONE 表示只操作目标用户，ALL 时 
# 表示目标用户的id 不再重要，会记录所有用户的歌曲之间的相似度
# 同时ALL时 是将数据存储到song 中，记录为SimilaritySongs
# ONE，是将数据存储到recom 中，记录为RecomSong
# mongomus.makeRecomBySongSimilarityAnswer(limit="ONE")
# 此处暂时使用ONE 标签的程序过程
# 此处sign 适用于判断是输出哪种格式
# mongomus.makeRecomAnswerSong(sign=1)
# 首先是ONE 和 ALL 两种模式的共同方法

# 获取recom中之前设置的数据
projections={"_id": 0, "id": 1, "name": 1, "recomUsers": 1}
doc=mongomus.rdb.findDocument(
        collection_name = "recom", query = mongomus.query, projection = projections)
# 储存请求用户的名字
mongomus.usr.user_name=doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id=[]
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
# print(doc)
list_users_id.extend(doc[0]['recomUsers'])
# 此处生成list_users_id
print(len(list_users_id))
for item in list_users_id:
    print(item)
    break


37
1a78ac10b67911ed95dd00155dadb10b


In [3]:
# 构成xy轴的内容源头
docs=mongomus.usr.getSongsWithUserIdFromMongo(array = list_users_id)

In [4]:
# xy稀疏矩阵信息,显示docs 的长度
# print(type(docs))
print(len(docs))


37


In [5]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data=mongomus.usr.makeXYMatric(docs = docs)
# data 存储的是每个歌曲信息的坐标位置信息
# for item in data:
#     print(data)
#     break
# 生成pandas
sudf=supd.SUPandas(datas = list(
    map(list, data.toarray())), items = mongomus.usr.y, users = mongomus.usr.x)

In [6]:
# 最后构成的稀疏矩阵
sudf.df

,RuRuUIH,WfTR2,H298YN,mdOQjX,G,X2Bwh38,9D0ZA,p,UnfVc1Tg,H6fLkSvrlg,...,S0JlP3z,KvC,zVZzhaw,1C3ijya,IDDRfNJpw,D0nHKfvH,FnJi,GcTaF8d,afqmIx79w,dj775w
491529,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
567083018,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
29097995,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
29196302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516358164,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326503,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1540081,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400162805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1540090,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# 寻找用户的推荐歌曲
projections = {"_id": 0, "songs": 1}
# dict_song = mongomus.makeRecomSongDictionForUser(
#     df_object=sudf, user_songs=mongomus.rdb.findDocument(
#         collection_name="like", query=mongomus.query, projection=projections)[0]['songs'])
# 获得目标用户的songs
user_songs = mongomus.rdb.findDocument(collection_name="like", query=mongomus.query, projection=projections)
# 展示第一条数据
print(user_songs[0]['songs'][0])
user_songs = user_songs[0]['songs']

{'id': 1370027704, 'name': '复甦（中式医乐·黄钟宫五正调）', 'union': [{'tags': ['轻音乐', '浪漫', '经典']}]}


In [8]:
similar_item = sudf.makeSimilarityBetweenItem()
similar_user = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [9]:
similar_item

,491529,567083018,29097995,29196302,516358164,28377108,516358165,1456603168,1605671,26116136,...,4358101,1476132827,28868577,32309218,1377501157,2326503,1540081,400162805,1540090,4063229
491529,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.25,0.0,0.00,0.0,0.333333,0.0,0.0,0.0,0.0
567083018,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.25,0.0,0.00,0.0,0.333333,0.0,0.0,0.0,0.0
29097995,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
29196302,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.5,0.000000,...,0.000000,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0
516358164,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.00,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326503,0.333333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.00,0.0,0.00,0.0,1.000000,0.0,0.0,0.0,0.0
1540081,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.333333,0.00,0.0,0.00,0.0,0.000000,1.0,0.0,0.0,0.0
400162805,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.00,0.0,0.00,0.0,0.000000,0.0,1.0,0.0,0.0
1540090,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.000000,0.00,0.0,0.20,0.0,0.000000,0.0,0.0,1.0,0.0


In [10]:
similar_user

,RuRuUIH,WfTR2,H298YN,mdOQjX,G,X2Bwh38,9D0ZA,p,UnfVc1Tg,H6fLkSvrlg,...,S0JlP3z,KvC,zVZzhaw,1C3ijya,IDDRfNJpw,D0nHKfvH,FnJi,GcTaF8d,afqmIx79w,dj775w
RuRuUIH,1.000000,0.010700,0.013115,0.016113,0.010799,0.015496,0.015459,0.015343,0.017824,0.008159,...,0.014299,0.011655,0.023783,0.011936,0.010979,0.003448,0.016895,0.013873,0.007716,0.015831
WfTR2,0.010700,1.000000,0.003958,0.014085,0.016008,0.014414,0.015306,0.011164,0.017399,0.010020,...,0.015993,0.011000,0.019474,0.012291,0.025472,0.004155,0.015611,0.014925,0.006321,0.012195
H298YN,0.013115,0.003958,1.000000,0.005502,0.007657,0.008584,0.009138,0.004745,0.007491,0.006861,...,0.012887,0.005111,0.008052,0.004141,0.006083,0.000000,0.001462,0.006723,0.005362,0.004082
mdOQjX,0.016113,0.014085,0.005502,1.000000,0.016473,0.015755,0.013055,0.017256,0.022184,0.017690,...,0.017286,0.019751,0.013958,0.008055,0.019247,0.004335,0.011278,0.017472,0.007895,0.020809
G,0.010799,0.016008,0.007657,0.016473,1.000000,0.018924,0.015829,0.016579,0.015343,0.008919,...,0.019391,0.019080,0.013963,0.015171,0.011484,0.006472,0.016211,0.009912,0.013158,0.008717
X2Bwh38,0.015496,0.014414,0.008584,0.015755,0.018924,1.000000,0.016071,0.016779,0.011217,0.013845,...,0.012313,0.010582,0.019547,0.011905,0.015332,0.013636,0.012536,0.014737,0.006803,0.009423
9D0ZA,0.015459,0.015306,0.009138,0.013055,0.015829,0.016071,1.000000,0.013460,0.019737,0.011905,...,0.020084,0.017910,0.014368,0.011013,0.019417,0.004087,0.016364,0.019763,0.006227,0.012035
p,0.015343,0.011164,0.004745,0.017256,0.016579,0.016779,0.013460,1.000000,0.018605,0.012026,...,0.015711,0.016682,0.019802,0.013279,0.015994,0.004957,0.017933,0.014679,0.006849,0.014213
UnfVc1Tg,0.017824,0.017399,0.007491,0.022184,0.015343,0.011217,0.019737,0.018605,1.000000,0.016393,...,0.021190,0.009551,0.013915,0.007407,0.021310,0.006527,0.015873,0.011385,0.010804,0.020213
H6fLkSvrlg,0.008159,0.010020,0.006861,0.017690,0.008919,0.013845,0.011905,0.012026,0.016393,1.000000,...,0.015733,0.009650,0.010417,0.004115,0.010348,0.009158,0.009751,0.008343,0.001610,0.008186


In [11]:
# 生成用于存储的结果
# mongomus.makeRecomSongDictionForUser(
#     df_object=sudf, user_songs=user_songs)

diction = sudf.makeTopNSongs(
    # 对用户收藏的歌曲，每首获取最相似的前5首
    #  将这个数据进行几个合并，
    # 然后将数据进行随机提取，
    # 提取其中的 前5个
    similar=similar_item,
    song_cnt=5)
# 此处显示获取到每首歌的相似度前5首歌曲
print(diction)

{491529: [567688074, 1427912609, 1874946820, 1449559810, 416011159], 567083018: [1379930956, 1465217165, 26395629, 29379917, 31719272], 29097995: [1608875, 468932689, 496298886, 25916549, 22733726], 29196302: [29539330, 5040736, 32823793, 1493618336, 34367784], 516358164: [29712208, 19089329, 29719536, 16330160, 451701130], 28377108: [554210, 459178, 1456185172, 3884125, 515835243], 516358165: [27609788, 494642794, 522529335, 36307308, 432698441], 1456603168: [429473, 34248838, 1817430967, 22514735, 1443503506], 1605671: [5114866, 28065912, 17690713, 5057823, 485992658], 26116136: [19639388, 20363319, 1832684671, 465180672, 522297477], 25657383: [5048241, 1356981576, 20958791, 17970293, 34834414], 622645: [32187040, 444416774, 559204807, 1325431892, 29719512], 23986232: [5058792, 1454382140, 1369926862, 5236635, 450041133], 31260730: [1223773, 36992342, 16349843, 1787637, 5405105], 16646207: [432509172, 1300595165, 32405499, 5052462, 27510434], 2195529: [1422766, 544526068, 2974141, 50

In [12]:
# 这个方法获取的是用户自自身收藏的歌曲的歌曲
list_songs = mongomus.makeSongIdList(user_songs)
print(len(list_songs))
print(list_songs)

448
[1370027704, 331160, 1324244054, 19153845, 510515495, 17358564, 183679, 29119025, 24615861, 507585627, 31192913, 5027852, 554024777, 525278875, 472716484, 19078789, 36226193, 488026985, 454698089, 553901186, 407677975, 28748595, 744932, 173466, 175246, 28977397, 29133639, 1306273863, 458356832, 28959185, 447578836, 1308782735, 499165825, 1356668748, 464797841, 404166555, 5057816, 571585964, 481014963, 22774088, 1461521, 528864917, 1311535641, 22498860, 5058719, 463031, 410181618, 26123237, 679645, 27074449, 22064605, 29418350, 20781240, 591071, 35504129, 27591589, 28785381, 2003591, 1325896815, 29100034, 1801563, 31370984, 16437022, 1476449694, 636692, 27506128, 4425889, 29418323, 495857412, 1059071, 21697173, 476515542, 204217, 442898049, 479813913, 256468, 18663058, 4007275, 28889785, 1693715, 2531854, 26584453, 420513233, 28838470, 26123464, 34045202, 5168414, 17594193, 19277298, 468998937, 29712208, 1883224, 27443402, 553715430, 413834666, 29284485, 529500547, 28240590, 1887366

In [13]:
# 此处运算通过删除用户已经收藏的歌曲，然后再随机获取得到结果
list_songs_result = mongomus.makeSongsFromSimilarDict(
    diction=diction, songs=list_songs)
print(len(list_songs_result))
print(list_songs_result)

30
[375917, 2004490, 1298700522, 515223355, 25731276, 4877892, 22063769, 29331390, 1873324242, 1376001637, 1317095952, 30814948, 523917932, 27922233, 661201, 1299383, 418602619, 537772844, 28267932, 20943688, 1935857, 92292, 22690160, 31452535, 4310342, 522320785, 21255965, 455382413, 1307334587, 420500691]


In [14]:
# 假设我们 LIMIT == "ALL"
#　一下是程序中启动语句
mongomus_all = mus.UserSongRecomAfterSongSimilarity()

# mongomus_all.saveUserSongRecomAnswer(
#                 mongomus_all.makeRecomAnswerSong(
#                     sign=0), update_name="SimilaritySongs", collection="song")


In [15]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "recomUsers": 1}
doc = mongomus_all.rdb.findDocument(
    collection_name="recom", query=mongomus_all.query, projection=projections)
# 储存请求用户的名字
mongomus_all.usr.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus_all.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
print(doc[0])
list_users_id.extend(doc[0]['recomUsers'])
# 构成xy轴的内容源头
docs = mongomus_all.usr.getSongsWithUserIdFromMongo(array=list_users_id)
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus_all.usr.makeXYMatric(docs=docs)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus_all.usr.y, users=mongomus_all.usr.x)

# 以上生成pandas 部分的代码基本一致
# 后面处理数据的方式不同，这里需要的是每首歌的相似歌曲，所以需要用户的信息


{'id': '1a78b6d8b67911ed95dd00155dadb10b', 'name': '2'}


KeyError: 'recomUsers'

In [ ]:
# 寻找相似歌曲
dict_song = mongomus_all.makeSimilaritySongDictionForSong(
    df_object=sudf)
# 该方法中内容
# diction = df_object.makeTopNSongs(
#             # 对用户收藏的歌曲，每首获取最相似的前5首
#             # 将这个数据进行几个合并，
#             # 然后将数据进行随机提取，
#             # 提取其中的 前5个
#             similar=df_object.makeSimilarityBetweenItem(),
#             song_cnt=5)

In [ ]:
print(dict_song)